In [15]:
import os
import time
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
import Turbine
from scipy.optimize import root

RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")

water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    RP=RP,
)

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel(
    "streams0.xlsx", sheet_name="water", index_col=0)
# таблица задаваемых значений (заменится на интерфейс)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)

KPD_SP = 0.99
calctolerance = 10**-3
start_time = time.time()


class turboustanovka:
    def __init__(self, water, water_streams0, water_streams, KPD_SP=0.99):
        self.Tepl_systema = SP.teplofik_systema(
            KPD_SP, water,  water_streams0, water_streams)

        self.Turb = Turbine.turbine(water,
                                    "DROSVD-TURBVD",
                                    "ENDOFVD",
                                    "DROSND-TURBND",
                                    "SMESHEND",
                                    "DOOTB2",
                                    "DOOTB1",
                                    "INCND",
                                    "INKOND",
                                    water_streams0,
                                    water_streams,
                                    )
        self.water_streams = water_streams

    def calculate_t_rejim(self, calcmethod, calctolerance, maxiterations):

        def Find_Potb2(diafragma):
            Turb_res = self.Turb.calculate(diafragma, maxiterations,
                                           calctolerance)
            Tepl_systema_res = self.Tepl_systema.calculate(
                maxiterations, calctolerance)
            Potb2_turb = self.water_streams.at['DOOTB2', 'P']
            Potb2_teplof = self.water_streams.at['OTB2-SP2', 'P']
            return ((Potb2_turb-Potb2_teplof)/Potb2_teplof)*100

        Solution_Potb2 = root(Find_Potb2, self.water_streams.at['OTB2-SP2', 'P'],
                              method=calcmethod, tol=calctolerance)
        Diafragma = float(Solution_Potb2.x)
        
        Turb_res = self.Turb.calculate(Diafragma, maxiterations,
                                           calctolerance)
        Tepl_systema_res = self.Tepl_systema.calculate(
                maxiterations, calctolerance)        
        
        Result = {'Turb_res': Turb_res, 'Tepl_systema_res': Tepl_systema_res, 'Diafragma': Diafragma}
        return Result

    def calculate(self, teplofikacia=0, calcmethod='hybr', calctolerance=10**-3, maxiterations=20,):

        if teplofikacia == 1:
            Result = self.calculate_t_rejim(
                calcmethod, calctolerance, maxiterations)

        if teplofikacia == 0:
            diafragma = 0
            Turb_res = self.Turb.calculate(diafragma, maxiterations,
                                           calctolerance)
            Result = {'Turb_res': Turb_res}
        return Result


start_time = time.time()
TU = turboustanovka(water, water_streams0, water_streams, KPD_SP)
TU_res = TU.calculate(teplofikacia=0, calcmethod='hybr',
                      calctolerance=10**-3, maxiterations=10)

print("fin:--- %s сек. ---" % round((time.time() - start_time), 1))

Максимальная погрешность определения давления в отборах 0.0005801064546580738
fin:--- 0.2 сек. ---


In [2]:
water_streams['SWIN-TURB':'INKOND']

,T,P,H,G,S,X
SWIN-TURB,69.000000,1.000000,NaN,783.300000,NaN,NaN
SWIN-OD,69.000000,1.000000,NaN,98.673017,NaN,NaN
OD-SP1,101.145109,1.000000,423.494571,98.673017,NaN,NaN
OTB2-SP2,116.100000,0.239735,2709.827925,22.692739,NaN,NaN
SP2-SP1,124.366793,0.227749,522.378652,22.692739,NaN,NaN
SP1-SP2,105.515957,1.000000,441.795311,783.300000,NaN,NaN
SP2-WOUT,120.500000,1.000000,504.533500,783.300000,NaN,NaN
WIN-SP1,73.049349,1.000000,192.602000,783.300000,NaN,NaN
OTB1-SP1,113.448857,0.160800,2696.262800,47.732553,NaN,NaN
DROSVD-TURBVD,492.149590,5.189023,3414.093682,41.970000,6.934922,100.000000
